# Sentence Quality Scorer

In [1]:
!pip install textacy
!python3 -m spacy download en
import textacy
import pandas as pd
from textacy.text_stats import TextStats
print("Loaded")

    100% |████████████████████████████████| 143kB 2.5MB/s ta 0:00:01
    100% |████████████████████████████████| 92kB 3.0MB/s ta 0:00:011
    100% |████████████████████████████████| 12.4MB 53kB/s  eta 0:00:01   71% |███████████████████████         | 8.9MB 21.1MB/s eta 0:00:01
    100% |████████████████████████████████| 51kB 6.0MB/s eta 0:00:01
    100% |████████████████████████████████| 1.6MB 439kB/s eta 0:00:01
    100% |████████████████████████████████| 2.0MB 369kB/s eta 0:00:01
    100% |████████████████████████████████| 235kB 3.0MB/s eta 0:00:01
    100% |████████████████████████████████| 31.2MB 20kB/s  eta 0:00:01   15% |████▉                           | 4.7MB 46.4MB/s eta 0:00:01    70% |██████████████████████▍         | 21.8MB 40.4MB/s eta 0:00:01
    100% |████████████████████████████████| 51kB 5.3MB/s eta 0:00:01
  Running setup.py bdist_wheel for pyemd ... done
  Stored in directory: /home/jovyan/.cache/pip/wheels/e4/ba/b0/1f4178a35c916b22fc51dc56f278125d4b8cfb0592e5f0cc24
  

    100% |████████████████████████████████| 37.4MB 62.0MB/s ta 0:00:01  5% |█▉                              | 2.1MB 54.2MB/s eta 0:00:01
  Running setup.py install for en-core-web-sm ... done
You are using pip version 9.0.3, however version 10.0.1 is available.
You should consider upgrading via the 'pip install --upgrade pip' command.

    Linking successful
    /srv/conda/lib/python3.6/site-packages/en_core_web_sm -->
    /srv/conda/lib/python3.6/site-packages/spacy/data/en

    You can now load the model via spacy.load('en')

Loaded


In [2]:
df = pd.read_csv("./abstraction_scored.csv")
df.clauses_text_final = df.clauses_text_final.apply(eval)
df.voice = df.voice.apply(eval)
df.abstraction_score = df.abstraction_score.apply(eval)
df.sample(frac = 1).head(10)

,survey_id,prompt_number,prompt_id,prompt,response,clauses_text_final,voice,idx,abstraction_score
6,1869,25,25,My main problem is,a feeling that I am not entitled,"[a feeling, that I am not entitled]","[Undefined, P_bevb_x]",6,"[0.14, 0.22]"
21,2381,34,47,Technology,"is a great tool, nice to have and can bring gl...","[is a great tool nice, to have and, can bring ...","[P_bevb_x, A_def, A_def, A_def, P_bevb_x, A_de...",21,"[0.25, 0.14, 0.25, 0.14, 0.12, 0.22, 0.14]"
42,1665,5,5,Being with other people,Is a mutually mutating meeting of universes,"[Is a meeting of universes, mutually mutating]","[P_bevb_x, A_def]",42,"[0.22, 0.14]"
1,2507,1,90,My family,has a cat...I don't have a cat though I have 2...,"[has a cat, I don t, have a cat, though I have...","[A_def, A_def, A_def, A_def]",1,"[0.25, 0.22, 0.25, 0.22]"
38,1823,6,6,The thing I like about myself is,that here I sit in the drop of this essential ...,"[D, that here I sit in the drop of this essent...","[Undefined, A_def, A_def]",38,"[0.0, 0.22, 0.25]"
4,3356,34,47,Technology,its crazy and i wish i could keep up,"[and i wish, i could keep up]","[A_def, A_def]",4,"[0.14, 0.25]"
45,2721,26,26,When I get mad,i feel adrenaline through my human body and it...,"[i feel adrenaline through my human body and, ...","[A_pron_x, P_bevb_x, A_def, A_def, P_bevb_x, A...",45,"[0.14, 0.25, 0.14, 0.25, 0.2, 0.25]"
8,1832,26,26,When I get mad,"it takes a long time to build up, but I am slo...","[it takes a long time but, to build up, I am s...","[A_def, A_def, P_bevb_x, A_def]",8,"[0.14, 0.12, 0.22]"
0,2546,36,48,Sometimes I wish that,my mom would come home more often.,[my mom would come home more often],[A_pron_x],0,[0.25]
10,1880,36,48,Sometimes I wish that,I could be younger and have done things differ...,"[I could be younger and, have done things diff...","[P_get_x, A_def]",10,"[0.25, 0.14]"


In [3]:
def score_readability(text):
    doc = textacy.Doc(text, lang = "en")
    ts = TextStats(doc)
    #print("############################################################")
    #print(text)
    #print("readability_stats = ", ts.readability_stats)
    return ts.readability_stats

df['readability_attributes_score'] = df.clauses_text_final.apply(lambda arr: [score_readability(x) for x in arr])
df['grading_level'] = df.readability_attributes_score.apply(lambda dct_arr: [round(dct['flesch_kincaid_grade_level'], 2) for dct in dct_arr])
df['reading_ease'] = df.readability_attributes_score.apply(lambda dct_arr: [round(dct['flesch_reading_ease'], 2) for dct in dct_arr])
_ = """[{'flesch_kincaid_grade_level': 0.6257142857142846, 'flesch_reading_ease': 103.04428571428573, 'smog_index': 3.1291, 'gunning_fog_index': 2.8000000000000003, 'coleman_liau_index': 2.6518669999999993, 'automated_readability_index': 0.23714285714285666, 'lix': 7.0, 'gulpease_index': 93.28571428571428, 'wiener_sachtextformel': -2.5074571428571426}]"""
del df['readability_attributes_score']
df.sample(frac = 1).head(10)

,survey_id,prompt_number,prompt_id,prompt,response,clauses_text_final,voice,idx,abstraction_score,grading_level,reading_ease
26,3099,22,43,At times I worry about,"big, big existential questions, and little mic...",[big big existential questions and little micr...,[Undefined],26,[0.25],[13.47],[23.43]
37,2619,2,2,When I am criticized,I can embody my personality and feel a pang co...,"[I can embody my personality and, feel a pang ...","[A_pron_x, A_def, A_def, A_def, A_def, A_def, ...",37,"[0.25, 0.25, 0.22, 0.12, 0.14, 0.25]","[8.38, 2.28, 4.45, 0.72, 3.65, 5.86]","[45.65, 92.97, 73.85, 97.03, 86.71, 72.62]"
44,1823,3,3,Change is,the heartbeat of the universe... aliveness cal...,"[the heartbeat of the universe aliveness, call...","[Undefined, A_def]",44,"[0.14, 0.25]","[4.45, 1.31]","[73.85, 90.99]"
4,3356,34,47,Technology,its crazy and i wish i could keep up,"[and i wish, i could keep up]","[A_def, A_def]",4,"[0.14, 0.25]","[-2.62, -2.23]","[119.19, 118.18]"
18,2409,5,73,Loving other people,can deepen with familiarity. Even with those w...,[can deepen with familiarity Even with those t...,"[A_def, A_def, A_def, A_def, A_def, A_def, P_b...",18,"[0.25, 0.14, 0.25, 0.14, 0.11, 0.14, 0.14, 0.2...","[9.93, -3.01, 5.86, 1.31, -3.01, 3.67, 2.89, 3...","[50.61, 120.21, 72.62, 90.99, 120.21, 75.88, 7..."
9,2390,17,17,When they avoided me,"I felt safe, knowing that I could receive a wa...","[I felt safe, that I could receive a warm welc...","[P_get_x, A_pron_x]",9,"[0.22, 0.25]","[-2.62, 4.82]","[119.19, 81.86]"
22,2344,12,12,A good boss,A good boss is critical toward making a workpl...,"[A good boss is critical toward, making a work...","[P_bevb_x, A_def]",22,"[0.25, 0.25]","[4.45, 10.74]","[73.85, 30.53]"
0,2546,36,48,Sometimes I wish that,my mom would come home more often.,[my mom would come home more often],[A_pron_x],0,[0.25],[0.63],[103.04]
11,2407,17,17,When they avoided me,I sought to find out 'why?' . Knowing the 'why...,"[I sought to find out, why Knowing, the why ca...","[A_def, A_def, A_def, A_def]",11,"[0.22, 0.14, 0.25]","[-1.84, 2.89, 4.0]","[117.16, 77.91, 78.87]"
43,2182,11,39,What I like to do best is,lay back and live the window that I have becom...,"[lay back and, live the window if a home with ...","[A_def, A_def, A_def, P_bevb_x, P_bevb_x, P_be...",43,"[0.25, 0.14, 0.25, 0.14, 0.14, 0.14, 0.14, 0.25]","[-2.62, 4.0, 0.72, 2.31, -1.45, 5.25, 0.72, 5.23]","[119.19, 89.52, 97.03, 90.96, 116.15, 62.79, 9..."


In [4]:
def normalize(row, x_max, x_min, reverse_arr = False):
    if not reverse_arr:
        return [round((x - x_min)/(x_max - x_min), 2) for x in row]
    return [round((-1*x - x_min)/(x_max - x_min), 2) for x in row]

reading_ease = df['reading_ease'].tolist()
reading_ease = [j for i in reading_ease for j in i]
reading_ease = [-1*x for x in reading_ease]
x_max, x_min = max(reading_ease), min(reading_ease)
df['reading_ease_normalized'] = df['reading_ease'].apply(lambda arr : normalize(arr, x_max, x_min, reverse_arr = True))

grading_levels = df['grading_level'].tolist()
grading_levels = [j for i in grading_levels for j in i]
x_max, x_min = max(grading_levels), min(grading_levels)
df['grading_level_normalized'] = df['grading_level'].apply(lambda arr : normalize(arr, x_max, x_min, reverse_arr = False))
df.sample(frac = 1).head(10)

,survey_id,prompt_number,prompt_id,prompt,response,clauses_text_final,voice,idx,abstraction_score,grading_level,reading_ease,reading_ease_normalized,grading_level_normalized
25,2276,30,30,If I were in charge,i would change that by facilitating change tha...,"[i would change that by, facilitating change i...","[A_def, A_pron_x, A_pron_x, A_def, P_bevb_x]",25,"[0.25, 0.25, 0.25, 0.25, 0.14]","[-1.84, 8.37, 9.82, 3.67, 0.63]","[117.16, 52.87, 53.12, 75.88, 103.04]","[0.02, 0.32, 0.32, 0.21, 0.09]","[0.05, 0.39, 0.44, 0.24, 0.13]"
3,2509,22,43,At times I worry about,"my own dreams, nightmares I have thousands of ...","[my own dreams nightmares, I have thousands of...","[Undefined, A_def]",3,"[0.14, 0.22]","[0.72, 0.52]","[97.03, 100.24]","[0.11, 0.1]","[0.14, 0.13]"
0,2546,36,48,Sometimes I wish that,my mom would come home more often.,[my mom would come home more often],[A_pron_x],0,[0.25],[0.63],[103.04],[0.09],[0.13]
39,1909,25,25,My main problem is,"integrating emptiness with everyday life, find...","[integrating emptiness with making, everyday l...","[A_def, A_def, A_def]",39,"[0.14, 0.14, 0.14]","[12.52, 10.35, 7.6]","[12.43, 31.55, 49.48]","[0.51, 0.42, 0.34]","[0.53, 0.46, 0.37]"
37,2619,2,2,When I am criticized,I can embody my personality and feel a pang co...,"[I can embody my personality and, feel a pang ...","[A_pron_x, A_def, A_def, A_def, A_def, A_def, ...",37,"[0.25, 0.25, 0.22, 0.12, 0.14, 0.25]","[8.38, 2.28, 4.45, 0.72, 3.65, 5.86]","[45.65, 92.97, 73.85, 97.03, 86.71, 72.62]","[0.36, 0.13, 0.22, 0.11, 0.16, 0.23]","[0.39, 0.19, 0.26, 0.14, 0.24, 0.31]"
23,2952,25,25,My main problem is,getting over decades spent proving myself.,"[getting over decades, spent proving myself]","[A_def, A_def, A_def]",23,"[0.25, 0.14]","[1.31, 5.25]","[90.99, 62.79]","[0.14, 0.27]","[0.16, 0.29]"
31,2690,28,83,A teacher has the right to,I don't mean to be oppositional. . but this se...,"[but, I don t mean to be oppositional, this se...","[Undefined, P_get_x, P_get_x, P_bevb_x, A_def,...",31,"[0.12, 0.22, 0.25, 0.25, 0.25, 0.22, 0.22, 0.2...","[-3.4, 4.32, 0.52, 7.37, 2.88, 1.31, -1.84, 6....","[121.22, 70.34, 100.24, 54.7, 83.32, 90.99, 11...","[0.0, 0.24, 0.1, 0.31, 0.18, 0.14, 0.02, 0.23,...","[0.0, 0.26, 0.13, 0.36, 0.21, 0.16, 0.05, 0.34..."
5,2537,26,26,When I get mad,I sometimes use big voice then I go somewhere ...,"[I sometimes use big voice, then I go somewher...","[A_def, P_get_x, A_def]",5,"[0.25, 0.22, 0.25]","[0.52, 2.88, 0.52]","[100.24, 83.32, 102.05]","[0.1, 0.18, 0.09]","[0.13, 0.21, 0.13]"
15,2856,31,31,My father,taught me how to think critically and to love ...,"[taught me how to think critically and, to lov...","[A_def, A_def, A_def]",15,"[0.25, 0.11]","[4.0, -2.62]","[78.87, 119.19]","[0.2, 0.01]","[0.25, 0.03]"
21,2381,34,47,Technology,"is a great tool, nice to have and can bring gl...","[is a great tool nice, to have and, can bring ...","[P_bevb_x, A_def, A_def, A_def, P_bevb_x, A_de...",21,"[0.25, 0.14, 0.25, 0.14, 0.12, 0.22, 0.14]","[-1.84, -2.62, 9.66, 9.18, 6.42, 5.24, 0.52]","[117.16, 119.19, 40.09, 34.59, 59.75, 66.4, 10...","[0.02, 0.01, 0.38, 0.41, 0.29, 0.26, 0.09]","[0.05, 0.03, 0.44, 0.42, 0.33, 0.29, 0.13]"


In [6]:
#Cross verify that they are correct
reading_ease = df['reading_ease_normalized'].tolist()
reading_ease = [j for i in reading_ease for j in i]
grade = df['grading_level_normalized'].tolist()
grade = [j for i in grade for j in i]
print(max(reading_ease), min(reading_ease), max(grade), min(grade))
df[["prompt", "response", "clauses_text_final", "voice", "idx", "abstraction_score", "reading_ease_normalized", "grading_level_normalized"]].to_csv("readability_scored.csv", index = False)

1.0 0.0 1.0 0.0


In [25]:
from textacy.keyterms import sgrank
df['nlp_doc'] = df.apply(lambda row : textacy.Doc(row['prompt'] + " " + row['response'], lang = "en"), axis = 1)

df['sgrank'] = df['nlp_doc'].apply(lambda doc : sgrank(doc, n_keyterms = len(doc)))


Sometimes I wish that my mom would come home more often. []
My family has a cat...I don't have a cat though I have 2 dogs [('-PRON- family', 0.5152073053670754), ('cat', 0.2872958994527361), ('dog', 0.19749679518018837)]
When I am nervous my hands get shaky [('nervous -PRON- hand', 0.649122638064138), ('shaky', 0.3508773619358619)]
At times I worry about my own dreams, nightmares I have thousands of them [('thousand', 0.4085196694311275), ('nightmare', 0.28112896883965677), ('dream', 0.1860407949214892), ('time', 0.12431056680772645)]
Technology its crazy and i wish i could keep up []
When I get mad I sometimes use big voice then I go somewhere quiet so I can cry by myself. [('big voice', 0.5571203378752521), ('quiet', 0.23502916354253695), ('mad', 0.20785049858221094)]
My main problem is a feeling that I am not entitled [('-PRON- main problem', 0.649122638064138), ('feeling', 0.3508773619358619)]
If my mother was a different lady, I would not be who I am. [('different lady', 0.6491226

0     12
1     17
2      8
3     15
4     10
5     21
6     11
7     16
8     20
9     22
10    14
11    24
12    16
13    14
14    27
15    13
16    20
17    28
18    58
19    15
20    46
21    42
22    17
23    11
24    26
25    50
26    23
27    34
28    29
29    15
30    39
31    83
32    21
33    15
34    74
35    56
36    37
37    51
38    25
39    21
40    21
41    20
42    11
43    56
44    13
45    35
46    23
47    52
48    13
Name: nlp_doc, dtype: int64